In [3]:
import nltk
from nltk.corpus import stopwords
import stanza

# Here instead of narrowsing the sentiment of aspects into positive or negative,
#we are returning the sentiment word and leaving the choice to the user itself for judging the sentiment of the aspect.
#( Clearly shown in the output)  
def aspect_sentiment_analysis(txt, stop_words, nlp):
    
    txt = txt.lower()  # Lowercasing the given Text
    sentList = nltk.sent_tokenize(txt)  # Splitting the text into sentences

    fcluster = []
    dic = {}

    for line in sentList:
        txt_list = nltk.word_tokenize(line)  # Splitting up into words
        taggedList = nltk.pos_tag(txt_list)  # Doing Part-of-Speech Tagging to each word

        doc = nlp(line)  # Object of Stanza NLP Pipeline
        
        # Getting the dependency relations between the words
        dep_node = []
        for dep_edge in doc.sentences[0].dependencies:
            dep_node.append([dep_edge[2].text, dep_edge[0].id, dep_edge[1]])

        # Converting it into an appropriate format
        for i in range(0, len(dep_node)):
            if (int(dep_node[i][1]) != 0):
                dep_node[i][1] = txt_list[(int(dep_node[i][1]) - 1)]

        featureList = []
        for i in taggedList:
            if i[1].startswith('JJ'):  # Filter adjectives
                featureList.append(i[0])

        for i in featureList:
            filist = []
            for j in dep_node:
                if ((j[0] == i or j[1] == i) and j[2] in ["nsubj", "acl:relcl", "obj", "dobj", "agent", "advmod", "amod", "neg", "prep_of", "acomp", "xcomp", "compound"]):
                    if j[0] == i:
                        filist.append(j[1])
                    else:
                        filist.append(j[0])
            fcluster.append([i, filist])

    for i in fcluster:
        aspect = i[0]
        related_adjectives = ' '.join(i[1]).replace(' ', '')  # Combine words and remove spaces

        # Check for negation and adjust sentiment
        if "not" in i[1]:
            aspect_tokens = txt.split()
            not_index = i[1].index("not")
            if not_index < len(aspect_tokens) - 1:
                next_word = aspect_tokens[not_index + 1]
                aspect = "not_" + aspect if next_word in stop_words else "not " + aspect
            related_adjectives = related_adjectives.replace("not", "")

        if aspect not in dic:
            dic[aspect] = related_adjectives
        else:
            dic[aspect] += ' ' + related_adjectives
            
    finalcluster = [[value, [key]] for key, value in dic.items()]
    return finalcluster

Product_Review = """
The new smartphone is fantastic. The camera quality is excellent, capturing sharp and clear photos. 
The battery life exceeds expectations, lasting all day with regular use. 
The sleek design and vibrant display make it a pleasure to use. However,it has high price.
"""

nlp = stanza.Pipeline()
stop_words = set(stopwords.words('english'))
print(aspect_sentiment_analysis(Product_Review, stop_words, nlp))

2023-12-10 23:19:49 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-12-10 23:19:51 INFO: Loading these models for language: en (English):
| Processor    | Package                   |
--------------------------------------------
| tokenize     | combined                  |
| mwt          | combined                  |
| pos          | combined_charlm           |
| lemma        | combined_nocharlm         |
| constituency | ptb3-revised_charlm       |
| depparse     | combined_charlm           |
| sentiment    | sstplus                   |
| ner          | ontonotes-ww-multi_charlm |

2023-12-10 23:19:51 INFO: Using device: cpu
2023-12-10 23:19:51 INFO: Loading: tokenize
2023-12-10 23:19:51 INFO: Loading: mwt
2023-12-10 23:19:51 INFO: Loading: pos
2023-12-10 23:19:51 INFO: Loading: lemma
2023-12-10 23:19:51 INFO: Loading: constituency
2023-12-10 23:19:51 INFO: Loading: depparse
2023-12-10 23:19:51 INFO: Loading: sentiment
2023-12-10 23:19:51 INFO: Loading: ner
2023-12-10 23:19:52 INFO: Done loading processors!


[['smartphone', ['new']], ['smartphone', ['fantastic']], ['quality', ['excellent']], ['photos', ['sharp']], ['', ['clear']], ['use', ['regular']], ['design', ['sleek']], ['display', ['vibrant']], ['price', ['high']]]
